# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store quantum circuits, experiments, build experiment, and excute them on local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy on local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install qurrium
```


In [1]:
from IPython.display import display
from qurry import __version__

print(f"| Current Version: {__version__}")

| Current Version: 0.10.0


/home/harui/anaconda3/envs/quantumsphere12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.1 Setup Environment and Creat experiment executor


In [2]:
from qurry import EntropyMeasure
from qiskit import QuantumCircuit

- Import simulator


In [3]:
from qiskit.providers.basic_provider import BasicProvider

basic_provider = BasicProvider()
backend_sim = basic_provider.get_backend("basic_simulator")

- Next, we will initialize our `EntropyMeasure` instance. There are two methods implemented based on two theories. The defaul method is `randomized` based on Randomized Measure. The other method is `hadamard` based on Hadamard Test. 

- In the following tutorial, we will use `randomized` as our measurement method.


In [4]:
experiment_hadamard = EntropyMeasure(method="hadamard")
print('| The first method is which is `Hadamard Test`".')
experiment_randomized = EntropyMeasure()
print("| The default method is which is `Randomized Measure` we will use.")

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuits

- In `Qurry`, there are built-in quantum circuits that construct certain wave functions, or equivalently, quantum states. They can be imported from `qurry.recipe`.
- The user can define customized circuits following Qiskit workflow.



In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘


In [7]:
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)

| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                                        └───┘


In [8]:
sample03 = QuantumCircuit(8)
sample03.x(range(0, 8, 2))
print("| Custom circuit:")
print(sample03)

| Custom circuit:
     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     └───┘
q_5: ─────
     ┌───┐
q_6: ┤ X ├
     └───┘
q_7: ─────
          


- After we prepare three quantum circuits, we append them to the `EntropyMeasure` object by the `.add()` method.
- The attribute `.waves` of `Qurry` object is a customized `dict`. we can check the stored circuits by printing the dictionary.

In [9]:
experiment_randomized.add(
    wave=sample01,  # The circuit object
    key="TrivialParamagnet",  # The name of the circuit
    replace=False,  # True for if you want to replace the circuit with the same name.
)  # Default is False, so if you add the same name when False,
# it will be added with serial number.
experiment_randomized.add(sample02, "GHZ")
experiment_randomized.add(sample03)
print(experiment_randomized.waves)

{'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7a3f3026a210>, 'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x7a3f301faa20>, 2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7a3f300c4fe0>}


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.


## 1.3 Excute Circuits


- We can use `.measure()` method to excute the specific circuit in the `EntropyMeasure` object. The `.measure()` method  returns a hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- `EntropyMeasure.measure()` requires the following input parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurements
  - shots: the number of shots for each measurement. The default is 1024.


In [10]:
exp1 = experiment_randomized.measure(
    wave="TrivialParamagnet", times=100, shots=1024, backend=backend_sim
)
exp1

'e2e724f3-b5e1-479a-9c07-7c9b3725408d'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`


In [11]:
exp2 = experiment_randomized.measure(
    wave="TrivialParamagnet",
    times=100,
    shots=1024,
)
exp2

'fe422c6a-6730-4542-be55-7b6e7f991003'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.


In [12]:
print(experiment_randomized.exps)

{'e2e724f3-b5e1-479a-9c07-7c9b3725408d': <EntropyMeasureRandomizedExperiment(exp_id=e2e724f3-b5e1-479a-9c07-7c9b3725408d, EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, qubits_measured=[0, 1, 2, 3, 4, 5, 6, 7], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, actual_num_qubits=8, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7], random_unitary_seeds=None), Commonparams(exp_id='e2e724f3-b5e1-479a-9c07-7c9b3725408d', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x7a3f301d0530>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-12-09 14:23:34', 'run.001': '2024-12-09 14:23:34'})), unused_args_num=0, analysis_num=0)>, 'fe422c6a-6730-4542-be55-7b6e7f991003': <EntropyMeasureRandomizedExperiment(exp_

- The following is the other informatiom you can access.


In [13]:
experiment_randomized.waves

WaveContainer({
  'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7a3f3026a210>,
  'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x7a3f301faa20>,
  2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7a3f300c4fe0>})

In [14]:
experiment_randomized.exps[exp1]

<EntropyMeasureRandomizedExperiment(exp_id=e2e724f3-b5e1-479a-9c07-7c9b3725408d, 
  EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, qubits_measured=[0, 1, 2, 3, 4, 5, 6, 7], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, actual_num_qubits=8, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7], random_unitary_seeds=None),
  Commonparams(exp_id='e2e724f3-b5e1-479a-9c07-7c9b3725408d', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x7a3f301d0530>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-12-09 14:23:34', 'run.001': '2024-12-09 14:23:34'})),
  unused_args_num=0,
  analysis_num=0))>

In [15]:
print(experiment_randomized.exps[exp1].statesheet())

 # EntropyMeasureRandomizedExperiment with exp_id=e2e724f3-b5e1-479a-9c07-7c9b3725408d
 - arguments
   - exp_name --------------------- experiment.N_U_100.qurrent_randomized
   - times ------------------------ 100
   - qubits_measured -------------- [0, 1, 2, 3, 4, 5, 6, 7]
   - registers_mapping ------------ {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
   - actual_num_qubits ------------ 8
   - unitary_located -------------- [0, 1, 2, 3, 4, 5, 6, 7]
   - random_unitary_seeds --------- None
 - commonparams
   - exp_id ----------------------- e2e724f3-b5e1-479a-9c07-7c9b3725408d   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - target_keys ------------------ ['TrivialParamagnet']
   - shots ------------------------ 1024
   - backend ---------------------- <qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x7a3f301d0530>
   - run_args --------------------- {}
   - transpile_args --------------- {}
   - tags ---------------

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.


---

## 1.4 Post-Processing

- To obtain the Renyi Entropy of the whole system and the subsystems. We use `exps[].analyze()` method to calculate and store a `Analysis` object in `Experiment`.
- The `analyze` method requires the following input parameters:
  - degree: The size or selected qubits of the subsystem. Please refer to the following chart for the selection.


### Qubit Selction Hint

For example, consider a system with 12 qubits indexed as `[0, 1, ... 11]`.

Entering an index and running `.analyze` will select the bitstring corresponding to the chosen qubit.

In [16]:
exp1_report1 = experiment_randomized.exps[exp1].analyze(selected_qubits=[3, 4])
# The subsystem of qubits 3 and 4 is selected for the analysis.
exp1_report2 = experiment_randomized.exps[exp1].analyze(selected_qubits=[0])
# The subsystem of qubit 0 is selected for the analysis.

| Preparing error mitigation of selected qubits: [3, 4] - 00:00 < 00:00
| Preparing error mitigation of selected qubits: [0] - 00:00 < 00:00                                                          


- The `analyze` will return the result of this analysis.


In [17]:
exp1_report2.export()

({'purity': np.float64(0.9748982810974121),
  'entropy': np.float64(0.03667639603724389),
  'puritySD': np.float64(0.42819715436570643),
  'entropySD': np.float64(0.6336639658762965),
  'num_classical_registers': 8,
  'classical_registers': [0],
  'classical_registers_actually': [0],
  'all_system_source': "AnalysisHeader(serial=0, datetime='2024-12-09 14:23:36', summoner=None, log={})",
  'purityAllSys': np.float64(1.2068503570556641),
  'entropyAllSys': np.float64(-0.27124680076759133),
  'puritySDAllSys': np.float64(1.4692747673931916),
  'entropySDAllSys': np.float64(1.7564028615717384),
  'num_classical_registers_all_sys': 8,
  'classical_registers_all_sys': None,
  'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7],
  'errorRate': np.float64(-0.09893654727034654),
  'mitigatedPurity': np.float64(0.8932378960188764),
  'mitigatedEntropy': np.float64(0.16288363544357304),
  'counts_num': 100,
  'taking_time': 0.001397791,
  'taking_time_all_sys': 0.019524198,
  'count

In [18]:
print(exp1_report1)
print(exp1_report1.header)

EntropyMeasureRandomizedAnalysis with serial=0, AnalysisInput(num_qubits=8, selected_qubits=[3, 4], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, shots=1024, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7]), AnalysisContent(purity=1.0587331390380859, entropy=-0.08233899395432488, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-12-09 14:23:36', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.


In [19]:
print(experiment_randomized.exps[exp1].reports)
print(experiment_randomized.exps[exp1].reports[0])
print(experiment_randomized.exps[exp1].reports[0].header)

AnalysisContainer(length=2, {0{...}, 1{...}})
EntropyMeasureRandomizedAnalysis with serial=0, AnalysisInput(num_qubits=8, selected_qubits=[3, 4], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, shots=1024, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7]), AnalysisContent(purity=1.0587331390380859, entropy=-0.08233899395432488, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-12-09 14:23:36', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.


In [20]:
main, side_prodct = experiment_randomized.exps[exp1].reports[0].export()

- And there is it


In [21]:
main

{'purity': np.float64(1.0587331390380859),
 'entropy': np.float64(-0.08233899395432488),
 'puritySD': np.float64(0.7521095061924191),
 'entropySD': np.float64(1.024870776950543),
 'num_classical_registers': 8,
 'classical_registers': [3, 4],
 'classical_registers_actually': [3, 4],
 'all_system_source': 'independent',
 'purityAllSys': np.float64(1.2068503570556641),
 'entropyAllSys': np.float64(-0.27124680076759133),
 'puritySDAllSys': np.float64(1.4692747673931916),
 'entropySDAllSys': np.float64(1.7564028615717384),
 'num_classical_registers_all_sys': 8,
 'classical_registers_all_sys': None,
 'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7],
 'errorRate': np.float64(-0.09893654727034654),
 'mitigatedPurity': np.float64(0.9196687073728206),
 'mitigatedEntropy': np.float64(0.12081384274684406),
 'counts_num': 100,
 'taking_time': 0.001110913,
 'taking_time_all_sys': 0.019524198,
 'counts_used': None,
 'input': {'num_qubits': 8,
  'selected_qubits': [3, 4],
  'registers_

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.


In [22]:
side_prodct

{'purityCells': {61: 1.2111549377441406,
  50: 0.32209205627441406,
  44: 0.3797760009765625,
  70: 0.2613182067871094,
  35: 0.47393798828125,
  58: 0.7015953063964844,
  18: 1.1573734283447266,
  4: 1.1942806243896484,
  20: 1.4656600952148438,
  30: 0.8432960510253906,
  41: 0.44956207275390625,
  17: 1.0634574890136719,
  22: 1.079324722290039,
  65: 0.5963325500488281,
  23: 0.5790805816650391,
  1: 0.4315605163574219,
  68: 0.35285377502441406,
  14: 1.228464126586914,
  33: 0.4017486572265625,
  74: 1.6256542205810547,
  47: 0.7844734191894531,
  46: 0.8800086975097656,
  34: 0.4720497131347656,
  78: 0.9330635070800781,
  80: 1.9316177368164062,
  6: 0.6825065612792969,
  76: 0.3419189453125,
  81: 0.5657768249511719,
  89: 1.3995018005371094,
  29: 0.30428504943847656,
  92: 2.554250717163086,
  93: 2.5583534240722656,
  97: 1.852487564086914,
  36: 0.5589103698730469,
  12: 0.4041404724121094,
  43: 0.8158187866210938,
  96: 0.5179290771484375,
  98: 1.039093017578125,
  99: 

In [23]:
experiment_randomized.exps[exp1].reports[0].statesheet()

 # EntropyMeasureRandomizedAnalysis with serial=0
 - header
   - serial -------------------------------------- 0
   - datetime ------------------------------------ 2024-12-09 14:23:36
   - summoner ------------------------------------ None
   - log ----------------------------------------- {}
 - input
   - num_qubits ---------------------------------- 8
   - selected_qubits ----------------------------- [3, 4]
   - registers_mapping --------------------------- {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
   - shots --------------------------------------- 1024
   - unitary_located ----------------------------- [0, 1, 2, 3, 4, 5, 6, 7]
 - outfields ----------------------------------- 0 ........  # Number of unused arguments.
 - content
   - purity -------------------------------------- 1.0587331390380859
   - entropy ------------------------------------- -0.08233899395432488
   - puritySD ------------------------------------ 0.7521095061924191
   - entropySD -------------------------

---

## 1.5 Export Current Result

Also, Qurry provides a method to export all results as multiple files in a folder.
Just use the method `write` of each `Experiment` instance.

In [24]:
experiment_randomized.exps[exp1].write()

('e2e724f3-b5e1-479a-9c07-7c9b3725408d',
 {'folder': 'experiment.N_U_100.qurrent_randomized.001',
  'qurryinfo': 'experiment.N_U_100.qurrent_randomized.001/qurryinfo.json',
  'args': 'experiment.N_U_100.qurrent_randomized.001/args/experiment.N_U_100.qurrent_randomized.001.id=e2e724f3-b5e1-479a-9c07-7c9b3725408d.args.json',
  'advent': 'experiment.N_U_100.qurrent_randomized.001/advent/experiment.N_U_100.qurrent_randomized.001.id=e2e724f3-b5e1-479a-9c07-7c9b3725408d.advent.json',
  'legacy': 'experiment.N_U_100.qurrent_randomized.001/legacy/experiment.N_U_100.qurrent_randomized.001.id=e2e724f3-b5e1-479a-9c07-7c9b3725408d.legacy.json',
  'tales.unitaryOP': 'experiment.N_U_100.qurrent_randomized.001/tales/experiment.N_U_100.qurrent_randomized.001.id=e2e724f3-b5e1-479a-9c07-7c9b3725408d.unitaryOP.json',
  'tales.randomized': 'experiment.N_U_100.qurrent_randomized.001/tales/experiment.N_U_100.qurrent_randomized.001.id=e2e724f3-b5e1-479a-9c07-7c9b3725408d.randomized.json',
  'reports': 'exper

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.


In [25]:
from qurry.process import AVAIBILITY_STATESHEET

print(AVAIBILITY_STATESHEET)

 | Qurry version: 0.10.0
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes   
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Error 
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct .............................. Yes    No     Yes   
   - dummy .................................. Yes    No     Yes   
   - test ................................... Yes    No     Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
--------------------------------------------------------
   + Yes ...... Working normally

In [26]:
from qurry.tools.qiskit_version import QISKIT_VERSION_STATESHEET

print(QISKIT_VERSION_STATESHEET)

 | Qurry version: 0.10.0
--------------------------------------------
 ### Qiskit version
 - main
 - deprecated
 - into-community
--------------------------------------------

